# Sources for this notebook
This is based on [this book - Learning Spark](https://learning.oreilly.com/library/view/learning-spark-2nd/9781492050032/)

Data used in here is available from books [website](https://github.com/databricks/LearningSparkV2.git)

Spark API [documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html)

Spark [Guides](https://spark.apache.org/docs/latest/index.html#where-to-go-from-here)

Here is the [sample paper](https://databricks-prod-cloudfront.cloud.databricks.com/public/793177bc53e528530b06c78a4fa0e086/0/6221173/100020/latest.html) from 2018. This is from dataricks website, but is irrelevant today.  In this notebook, I have translated one of the scala question to pyspark.

Focus of Databricks exam is dataframe and SQL api. Exam is 80% (or more) is based on code questions. 

Here is my [badge](https://credentials.databricks.com/28181734-274d-4085-89de-0fcb9b2824f8)

# Setup
Install Java (I am on 11), Spark and Python (I have 3.8). 

Install Jupyter server and jupyter plugin in vscode.

Set env variables as below (Change values according to your setup)

```
export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64
export SPARK_HOME=/home/kamal/spark-3.1.2-bin-hadoop3.2
export PATH=$PATH:$SPARK_HOME/bin:$SPARK_HOME/sbin
export PYTHONPATH=${SPARK_HOME}/python/:$(echo ${SPARK_HOME}/python/lib/py4j-*-src.zip):${PYTHONPATH}
```


# Create Sessions
## Create Spark Session

In [ ]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
.appName("airline").getOrCreate())


## Create Delta spark session
* This requires delta lake jar be downloaded from mavenrepository and placed in SPARK_HOME/jars. Restart Jupyter server after that.  
* Install delta-spark package (pip install delta-spark)

In [ ]:
from delta import *

builder = SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

delta_spark = configure_spark_with_delta_pip(builder).getOrCreate()


# Read and operate on Data - Departure DS
## Departure Delays - Read CSV with string schema
Data used here is available from databricks github [page](https://github.com/databricks/LearningSparkV2.git)

In [ ]:
csv_file = "./databricks-datasets/learning-spark-v2/flights/departuredelays.csv"
schema = "date string, delay int, distance int, origin string, destination string"
df = (spark.read
.schema(schema)
.option("header", "true")
.csv(csv_file)).cache()
df.printSchema()
df.first()

## Operate on Columns, with col

In [ ]:
from pyspark.sql.functions import *
try:
    df.select(col("distance") / 1000).show(2)
except:
    print("Operation with col failed")

try:
    df.select("distance" / 1000).show(2)
except:
    print("Operation without col failed")

try:
    df.select("distance").show(2)
except:
    print("select with just quotes failed")


## Using withColumn() to add new computed column

In [17]:
df.withColumn("calculatedCol", col("distance")/1000).show(2)

+--------+-----+--------+------+-----------+-------------+
|    date|delay|distance|origin|destination|calculatedCol|
+--------+-----+--------+------+-----------+-------------+
|01011245|    6|     602|   ABE|        ATL|        0.602|
|01020600|   -8|     369|   ABE|        DTW|        0.369|
+--------+-----+--------+------+-----------+-------------+
only showing top 2 rows



## Difference between distinct and drop_duplicate 
Drop duplicate accepts columns while distinct does not

In [31]:
print(df.select("origin", "destination").distinct().count())
print(df.drop_duplicates(["origin","destination"]).count())
try:
    print(df.drop_duplicates([col("origin"),col("destination")]).count())
except:
    print("dropDuplicates does not work with col function")    

4138


4138
dropDuplicates does not work with col function


## Timestamp from millisecond (since epoch)

In [ ]:
spark.sql("select timestamp_millis(1631163333673) as timestampdata").show(truncate=False)

## Working with JSON

### Extract with from_json()

In [ ]:
data = [(1, '''{"a": 1,"b": 2.3}''')]
schema = 'a INT, b FLOAT'
df_json = spark.createDataFrame(data, ("key", "value"))
df_json.select(from_json("value", schema)["a"].alias("id"), 
from_json(df_json.value, schema)["b"].alias("value")).show()

### Extract with get_json_object()

In [ ]:
df_json.select(get_json_object(df_json.value,"$.a").alias("id"), 
get_json_object("value","$.b").alias("value")).show()

## Filter Dataframe

In [24]:
df.printSchema()
from pyspark.sql.functions import col
try:
    df.select("distance","origin","destination").filter("distance > 1000").distinct().show(2)
except:
    print("Filter without col function works with SQL like clause does not work")

try:
    df.select("distance","origin","destination").filter("distance" > 1000).distinct().show(2)
except:
    print("Filter without col with only string quotes does not work")

try:
    df.select("distance","origin","destination").filter(col("distance") > 1000).distinct().show(2)
except:
    print("Filter with col with python syntax does not work")


root
 |-- date: string (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    1036|   DFW|        DCA|
|    2115|   EWR|        PDX|
+--------+------+-----------+
only showing top 2 rows

Filter without col with only string quotes does not work


+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    1036|   DFW|        DCA|
|    2115|   EWR|        PDX|
+--------+------+-----------+
only showing top 2 rows



# Create Views
* Global views are used, in queries, with "global_temp" prefix.

In [ ]:
df.createOrReplaceTempView("Airline")
df.createOrReplaceGlobalTempView("GlobalAirlineView")

In [ ]:
spark.sql("select * from Airline limit 2").show()

In [ ]:
spark.sql("""SELECT delay, origin, destination, 
              CASE
                  WHEN delay > 360 THEN 'Very Long Delays'
                  WHEN delay >= 120 AND delay <= 360 THEN 'Long Delays'
                  WHEN delay >= 60 AND delay < 120 THEN 'Short Delays'
                  WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
                  WHEN delay = 0 THEN 'No Delays'
                  ELSE 'Early'
               END AS Flight_Delays
               FROM Airline
               ORDER BY origin, delay DESC""").show(10)

In [ ]:
spark.sql("""SELECT *,
                CASE
                    WHEN delay < 0 THEN 'EARLY'
                    WHEN delay == 0 THEN 'ON-TIME'
                    ELSE 'DELAY'
                END AS FL_Delay
             FROM Airline""").show(6)   

# Add column to DF
## Use multiple conditions on column with when-otherwise

In [ ]:
from pyspark.sql.functions import *
df.withColumn('FL_delay', when(col("delay") > 0, 'DELAYED')
.when(col("delay") < 0, 'EARLY')
.otherwise('ON-TIME')).show(6)

# Set shuffle partitions

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions",4)
df.write.option("path","/home/kamal/airline_data").format("csv").saveAsTable("Airline")
df.rdd.getNumPartitions()

# Create, Use and drop databases

In [ ]:
spark.catalog.listDatabases()
spark.sql("drop database if exists newdb cascade")
spark.sql("create database newdb location '/home/kamal/spark-warehouse/newdatabase'")
spark.catalog.listDatabases()

In [ ]:
spark.sql("use newdb")

In [ ]:
df.write.saveAsTable("airline_date", mode="overwrite", format="parquet")

In [ ]:
spark.sql("select * from airline_date where origin = 'SFO'").createOrReplaceGlobalTempView("us_origin_airport_SFO_global")
spark.sql("select * from global_temp.us_origin_airport_SFO_global").count()

In [ ]:
spark.sql("show views").show()
spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global")

# Working with AVRO
Avro package is still external (but supported), so needs to be downloaded and placed in classpath. Download spark-avro jar from [mavenrepo](https://mvnrepository.com/artifact/org.apache.spark/spark-avro_2.12/3.1.2),  and place it in $SPARK_HOME/jars. Then restart Jupyter server 

In [ ]:
spark.conf.set("spark.jars.packages", "com.databricks:spark-avro_2.12:3.1.2")
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW episode_tbl
    USING com.databricks.spark.avro
    OPTIONS (
      path "./databricks-datasets/learning-spark-v2/flights/summary-data/avro/*"
    )
""")

In [ ]:
df_fl_summary = spark.sql("select * from episode_tbl")
df_fl_summary.count()
df_fl_summary.show(5)

# UDFs

In [ ]:
from pyspark.sql.types import LongType
import pandas as pd
def cubed(s: pd.Series) -> pd.Series:
    return s*s*s

df_range = spark.range(1,9)    
cubed_udf = pandas_udf(cubed, returnType=LongType())

In [ ]:
df_range.select("id",cubed_udf("id").alias("cubed")).show()

In [ ]:
spark.udf.register("cubed_udf",cubed_udf)

In [ ]:
df_range.createOrReplaceTempView("udf_test")
spark.sql("select id, cubed_udf(id) from udf_test").show()

# Delta Read and Write

In [ ]:
data = delta_spark.sql("select * from global_temp.GlobalAirlineView").cache()
data.write.format("delta").option("mergeSchema", "true").mode("overwrite").save("/tmp/delta-table/")

In [ ]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show(5)

# Spark Streaming
Spark streaming is not part of Databricks Spark exam. Do check syllabus time to time
## foreachBatch Function

foreachBatch lets us use "update", "append" and "complete" output modes. Whereas file sink only allows append.

In [ ]:
from functools import partial
def foreachBatchFunction(format, location, df, batchId):
        (df
        .write
        .mode("overwrite")
        .format(format)
        .save(location+str(batchId)))
writeMicroBatchUpdateToJSON = partial(foreachBatchFunction,"JSON","/home/kamal/jsonUpdate/")
writeMicroBatchAppendToJSON = partial(foreachBatchFunction,"JSON","/home/kamal/jsonAppend/")
writeMicroBatchCompleteToJSON = partial(foreachBatchFunction,"JSON","/home/kamal/jsonComplete/")
writeMicroBatchAppendToJSON10Sec = partial(foreachBatchFunction,"JSON","/home/kamal/jsonAppend10sec/")
writeMicroBatchToCSV = partial(foreachBatchFunction,"JSON","/home/kamal/sparkcsv/")

## Word Count
Generate test dat using [datastreamer](https://github.com/skamalj/datagenerator) - with followung config. It creates space separated list of  6 words in each line
>`lorem|words|textline|6`

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
sparkstream = (SparkSession.builder
.appName("airline").getOrCreate())
lines = (sparkstream
  .readStream.format("socket")
  .option("host", "localhost")
  .option("port", 4000)
  .load())

linesnojson = lines.select(get_json_object("value","$.textline").alias("line"))

words = linesnojson.select(explode(split(col("line"), " ")).alias("word"))
counts = words.groupBy("word").count()

checkpointDir = "/home/kamal/sparkcheckpoint"
checkpointFileSyncDir = "/home/kamal/sparkcheckpointcsv"

#streamingQuery = (counts
#  .writeStream
#  .format("console")
#  .outputMode("complete")
#  .trigger(processingTime="20 second")
#  .option("checkpointLocation", checkpointDir)
#  .start())

streamingQueryFile = (counts
  .writeStream
  .foreachBatch(writeMicroBatchToCSV)
  .outputMode("update")
  .trigger(processingTime="20 second")
  .option("checkpointLocation", checkpointFileSyncDir)
  .start())

In [ ]:
#streamingQuery.stop()
streamingQueryFile.stop()

## Streaming with eventtime
Use generator with following config
>`datatype|number|id|{"min":0,"max":10}`

>`datatype|float|temp|{"min":20,"max":50}`

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
sparkstream = (SparkSession.builder
.appName("avgtemp").getOrCreate())
readings = (sparkstream
  .readStream.format("socket")
  .option("host", "localhost")
  .option("port", 4000)
  .load().withColumn("current_timestamp", current_timestamp()))

readings_tab = readings.select(get_json_object("value","$.id").alias("id"),
get_json_object("value","$.temp").alias("temp"),
timestamp_seconds(round(get_json_object("value","$.eventtime") / 1000)).alias("eventtime"),
"current_timestamp")

readings_tab = readings_tab.withColumn("delayed_eventtime", col("eventtime") - expr("INTERVAL 10 seconds"))

eventcol = "delayed_eventtime"

samples = (readings_tab.withWatermark(eventcol, "10 seconds")
.groupBy("id", window(eventcol, "20 second"))
.agg(count("id").alias("samples"),avg("temp").alias("averageTemp"),
max("current_timestamp").alias("processed_at"), max(eventcol).alias("generated_at")))

## Cooment / uncomment below blocks as per need

#streamingQuery = (samples
#  .writeStream
#  .format("console")
#  .option("truncate", "false")
#  .outputMode("append")
#  .trigger(processingTime="10 second")
#  .start())
#
#streamingQueryJsonAppend = (samples
#  .writeStream
#  .foreachBatch(writeMicroBatchAppendToJSON10Sec)
#  .outputMode("append")
#  .option("checkpointLocation", "/home/kamal/spark/checkpoint/jsonappend10sec")
#  .trigger(processingTime="10 second")
#  .start())   

streamingQueryJsonAppend = (samples
  .writeStream
  .foreachBatch(writeMicroBatchAppendToJSON)
  .outputMode("append")
  .option("checkpointLocation", "/home/kamal/spark/checkpoint/jsonappend")
  .trigger(processingTime="20 second")
  .start())   

#streamingQueryJsonUpdate = (samples
#  .writeStream
#  .foreachBatch(writeMicroBatchUpdateToJSON)
#  .outputMode("update")
#  .option("checkpointLocation", "/home/kamal/spark/checkpoint/jsonupdate")
#  .trigger(processingTime="20 second")
#  .start())   

#streamingQueryJsonComplete = (samples
#  .writeStream
#  .foreachBatch(writeMicroBatchCompleteToJSON)
#  .outputMode("complete")
#  .option("checkpointLocation", "/home/kamal/spark/checkpoint/jsoncomplete")
#  .start())

## Stop Streaming Queries

In [ ]:
#streamingQueryJsonUpdate.stop()
streamingQueryJsonAppend.stop()

In [ ]:
dfc = spark.read.format("delta").load("/tmp/deltaeventscomplete").cache()
dfc.filter("id == 2").show(15, truncate = False)


In [ ]:
df = spark.read.format("delta").load("/tmp/deltaevents").cache()
df.filter("id == 2").show(15, truncate = False)

In [ ]:

from pyspark.sql.functions import col
(df.join(dfc,['id','window'], "fullOuter")
    .select("id", "window", dfc.samples.alias("complete"), df.samples,dfc.processed_at)
    .orderBy("id")
    .filter("id == 10")
    .show(50, truncate=False))

# People DS - Scala excercise from [Sample question](https://databricks-prod-cloudfront.cloud.databricks.com/public/793177bc53e528530b06c78a4fa0e086/0/6221173/100020/latest.html). Translated to Pyspark

## People  - Imports and Create DF

In [10]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#schema = StructType([
#  StructField("name", StringType(), True),
#  StructField("department", IntegerType(), True),
#  StructField("score", ArrayType(IntegerType()), True)
#])

# Works same as above
schema = "name string, department string, score array<int>"

peopleDF = spark.createDataFrame([
  ("Ali", 0, [100]),
  ("Barbara", 1, [300, 250, 100]),
  ("Cesar", 1, [350, 100]),
  ("Dongmei", 1, [400, 100]),
  ("Eli", 2, [250]),
  ("Florita", 2, [500, 300, 100]),
  ("Gatimu", 3, [300, 100])
],schema) 

In [11]:
peopleDF.schema

StructType(List(StructField(name,StringType,true),StructField(department,StringType,true),StructField(score,ArrayType(IntegerType,true),true)))

## Create windowSpec

In [ ]:
from pyspark.sql.window import Window

windowSpec = Window.partitionBy("department").orderBy(col("escore").desc())


## Execute using windowSpec

In [ ]:
people = (peopleDF.select("*", explode("score")
.alias("escore"))
.drop("score")
.select("name", expr("department *1.0").alias("Dept"),"escore", rank().over(windowSpec).alias("maxrank"))
.filter("maxrank == 1")
.drop("maxrank")
.orderBy("department", ascending=False)
.show(5))


## Explode and collect_list

In [12]:
people_listscore = peopleDF.select("name","department",explode("score").alias("score"))

In [13]:
people_listscore.groupBy("name").agg(collect_list("score")).show(20)

+-------+-------------------+
|   name|collect_list(score)|
+-------+-------------------+
| Gatimu|         [300, 100]|
|Barbara|    [300, 250, 100]|
|  Cesar|         [350, 100]|
|Florita|    [500, 300, 100]|
|    Eli|              [250]|
|Dongmei|         [400, 100]|
|    Ali|              [100]|
+-------+-------------------+



In [ ]:
mnmdata = "/home/kamal/Documents/dev/spark/LearningSparkV2/chapter2/py/src/data/mnm_dataset.csv"

mnmDF = (spark
.read
.format("csv")
.option("header","True")
.option("inferschema", "True")
.load(mnmdata))

resultDF = (mnmDF
.groupBy("State","Color")
.agg(sum("count").alias("Total"))
.orderBy("Total",ascending = False)
)

In [ ]:
resultDF.count()

# Show current spark context, get the UI url

In [ ]:
spark.sparkContext

# Working with string schema

In [ ]:
data = [[1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter",
"LinkedIn"]],
       [2, "Brooke","Wenig", "https://tinyurl.2", "5/5/2018", 8908, ["twitter",
"LinkedIn"]],
       [3, "Denny", "Lee", "https://tinyurl.3", "6/7/2019", 7659, ["web",
"twitter", "FB", "LinkedIn"]],
       [4, "Tathagata", "Das", "https://tinyurl.4", "5/12/2018", 10568, 
["twitter", "FB"]],
       [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web",
"twitter", "FB", "LinkedIn"]],
       [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568, 
["twitter", "LinkedIn"]]
      ]

schema = """`Id` INT, `First` STRING, `Last` STRING, `Url` STRING, \
`Published` STRING, `Hits` INT, `Campaigns` ARRAY<STRING>"""

In [ ]:
blogs = spark.createDataFrame(data,schema)
#blogs.write.format("json").save("blogs-json")

In [ ]:
blogs.withColumn("BigHitters", expr("Hits > 10000")).filter(col("First").endswith("i")).show()

# Fire Department Dataset excercise

### Load Dataset

In [ ]:
firecallsDF = (spark
.read
.format("csv")
.option("samplingRatio", .001)
.option("header", True)
.load("databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv"))

firecallsDF.select("CallType").distinct().show()

### String to Timestamp

In [ ]:
from pyspark.sql.functions import *
new_fire_df = (firecallsDF
.select("*",
    to_timestamp(col("CallDate"),"MM/dd/yyyy").alias("IncidentDate"),
    to_timestamp(col("WatchDate"), "MM/dd/yyyy").alias("OnWatchDate"),
    to_timestamp(col("AvailableDtTm"),"MM/dd/yyyy hh:mm:ss a").alias("AvailableDtTs"))
.where(col("Incidentdate").isNotNull())
.drop("CallDate")
.drop("WatchDate")
.drop("AvailableDtTm"))

### Which calltype generated most calls

In [ ]:
(firecallsDF
.select("CallType")
.filter(col("CallType").isNotNull())
.groupBy("CallType")
.agg(count("CallType").alias("count"))
.orderBy("count", ascending=False)
.show(30, truncate = False))

### What were all the different types of fire calls in 2018?

In [ ]:
(new_fire_df
.select("CallType",year("IncidentDate").alias("Year"))
.filter("Year == 2018")
.groupBy("CallType")
.agg(count("CallType").alias("count"))
.show(2)
)

In [ ]:
new_fire_df.columns

### Which neighborhood in San Francisco generated the most fire calls in 2018

In [ ]:
(new_fire_df
.select("city","Neighborhood","CallType",year("IncidentDate").alias("Year"))
.where("city in ('San Francisco','SF', 'SAN FRANCISCO')")
.where(col("Year") == 2018)  #Can also be written as "Year == 2018"
.where(col("CallType").like('%Fire%'))
.groupBy("Neighborhood")
.agg(count("Neighborhood").alias("TotalCalls"))
.orderBy("TotalCalls",ascending=False)
.show(3, truncate=False))

# UDF vs Pandas UDF

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import LongType

def cube(s):
    return s*s*s

spark.udf.register("cube", cube, LongType())

spark.range(1,5).createOrReplaceTempView("udf_test")

In [ ]:
spark.sql("select id, cube(id) from udf_test").show()

In [ ]:
cubed_udf = pandas_udf(cube, LongType())
df_id = spark.range(1,4)
df_id.select("id",cubed_udf("id")).show()

In [ ]:
from pyspark.sql.functions import expr
tripdelaysFilePath = "databricks-datasets/learning-spark-v2/flights/departuredelays.csv"
airportsnaFilePath =  "databricks-datasets/learning-spark-v2/flights/airport-codes-na.txt"
  
# Obtain airports data set
airportsna = (spark.read
  .format("csv")
  .options(header="true", inferSchema="true", sep="\t")
  .load(airportsnaFilePath))

airportsna.createOrReplaceTempView("airports_na")

departureDelays = (spark.read
  .format("csv")
  .options(header="true")
  .load(tripdelaysFilePath))

departureDelays = (departureDelays
.withColumn("delay", expr("CAST(delay as INT) as delay"))
.withColumn("distance", expr("CAST(distance as INT) as distance")))

departureDelays.createOrReplaceTempView("departureDelays")

# SQL Collection functions

In [25]:
peopleDF.createOrReplaceGlobalTempView("peopleDF")

In [26]:
spark.sql("select filter(score, s -> s = 250) as new_score from global_temp.peopleDF").show(truncate=False)

+---------+
|new_score|
+---------+
|[]       |
|[250]    |
|[]       |
|[]       |
|[250]    |
|[]       |
|[]       |
+---------+



In [ ]:
spark.sql("set -v").select("key","value").show(truncate=False)

# Partition vs Coalease

In [ ]:
df_big_range2 = spark.range(1000000).repartition(8)
df_big_range2.rdd.getNumPartitions()
df_big_range2.cache()
df_big_range2.count()

In [ ]:
df_big_range2.count()